In [1]:
# %load params_select.py
# import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"] = ""

import os

import seaborn as snb
from hyperopt import hp
from keras.initializers import glorot_uniform
import matplotlib.pyplot as plt
import uuid
from data_prepare import *
from index_components import zz500_t10
from objective import objective
from trial import run_a_trial

snb.set()
from loss import *

try:
    import _pickle as pickle
except:
    import pickle

identity = 'model05'

lstm_space = {
    'time_steps': hp.choice('time_steps', [64]),
    'batch_size': hp.choice('batch_size', [64]),
    'epochs': hp.choice('epochs', [1000]),  # [100, 200, 500, 1000, 1500, 2000]

    # for class
    'activation_last': hp.choice('activation_last', ['softmax']),
    # for regression
    # 'activation_last': hp.choice('activation', [None, 'linear']),

    #
    'shuffle': hp.choice('shuffle', [False]),
    'loss_type': hp.choice('loss', ['categorical_crossentropy']), #, 'weighted_categorical_crossentropy']),

    'layer1': {
        'units': hp.choice('layer1_units', [128]),
        # 'relu', 'sigmoid', 'tanh', 'linear'
        'activation': hp.choice('layer1_activation', ['tanh']),
        'is_BN': hp.choice('layer1_is_BN', [True]),
    },
    'layer2': {
        'units': hp.choice('layer2_units', [128]),
        # 'relu', 'sigmoid', 'tanh', 'linear'
        'activation': hp.choice('layer2_activation', ['tanh']),
        'is_BN': hp.choice('layer2_is_BN', [True]),
    },
    'layer3': {
        'units': hp.choice('layer3_units', [128]),
        # 'relu', 'sigmoid', 'tanh', 'linear'
        # Loss turns into 'nan'
        # As far as I know, it's the combination of relu and softmax that causes numerical troubles,
        # as relu can produce large positive values corresponding to very small probabilities.
        # If you change your model to use, say, tanh instead of relu for the last dense layer,
        # the problem will go away.
        'activation': hp.choice('layer3_activation', ['tanh']),
        'is_BN': hp.choice('layer3_is_BN', [True]),
    },

    # 'lr': hp.loguniform('lr', np.log(0.000001), np.log(0.0001)),
    'lr': hp.choice('lr', [0.0001]),
    'dropout': hp.quniform('dropout', 0.4, 0.41, 0.1),
    'recurrent_dropout': hp.quniform('recurrent_dropout', 0.4, 0.41, 0.1),
    'kernel_initializer': hp.choice('kernel_initializer', [glorot_uniform(seed=123)]),
    'bias_initializer': hp.choice('bias_initializer', [glorot_uniform(seed=456)]),
}

# kline2_params = {
#     'window': 256,
# }
# params_list.append(kline2_params)
# func_list.append(feature_kline2)
#
# label_by_multi_ma_params = {
#     'window': [3, 5, 10]
# }
# params_list.append(label_by_multi_ma_params)
# func_list.append(label_by_multi_ma)

features_space = {
    # 'kline': {
    #     'window': hp.choice('kline_window', [[60]])
    # },
    'kline2': {
        'window': hp.choice('kline2_window', [256])
    },
    'ma': {
        'ma_list': hp.choice('ma_list', [[1, 2, 3, 5, 8, 13, 21]]),
        'window': hp.choice('ma_window', [[60]]),
        'price': hp.choice('price', ['close'])
    },
    # 'label_by_ma_price': {
    #     'window': hp.choice('label_window', [60]),
    #     'next_ma_window': hp.choice('next_ma_window', [3]),
    #     'quantile_list': hp.choice('quantile_list', [# [0, 0.1, 0.3, 0.7, 0.9, 1],
    #                                                  # [0, 0.2, 0.4, 0.6, 0.8, 1],
    #                                                  # [0, 0.15, 0.3, 0.7, 0.85, 1],
    #                                                  # [0, 0.15, 0.35, 0.65, 0.85, 1],
    #                                                  # [0, 0.3, 0.7, 1],
    #                                                  [0, 0.33, 0.66, 1],
    #                                                  # [0, 0.2, 0.8, 1],
    #                                                  # [0, 0.4, 0.6, 1],
    #                                                  # [0, 0.5, 1],
    #                                                  # [0, 0.45, 1],
    #                                                  # [0, 0.55, 1]
    #                                                 ])
    # },
    'label_by_multi_ma': {
        'window': hp.choice('label_window', [[3, 5, 10]])
    },
    'label': {
        'class_list': hp.choice('class_list', [[0.0, 1.0]])
    }
}

space = {
    'features': features_space,
    'lstm': lstm_space,
    'split_dates': ["2016-01-01", "2017-01-01"]
}


if __name__ == '__main__':
    # file_name = '../data/cs_market.csv'
    # ohlcv_list = get_data(file_name=file_name, stks=zz500_t10)

    zz500 = pickle.load(open('data/zz500.pkl', 'rb'))
    ohlcv_list = [zz500]

    # pickle_file = 'data/sz50_ohlcv.pkl'
    # ohlcv_list = get_pickle_data(pickle_file, [])

    function = "params_select"
    if identity == "":
        identity = str(uuid.uuid1())

    print("identity: {}".format(identity))
    namespace = function + '_' + identity

    namespace = os.path.join('./', namespace)
    if not os.path.exists(namespace):
        os.makedirs(namespace)

    with open(os.path.join(namespace, 'ohlcv_list.pkl'), 'wb') as f:
        pickle.dump(ohlcv_list, f)

    # # loss
    # loss = 'categorical_crossentropy'
    # loss = weighted_categorical_crossentropy5
    objective_func = partial(objective, ohlcv_list=ohlcv_list, namespace=namespace)

    trials_file = os.path.join(namespace, 'trials.pkl')

    # while True:
    run_a_trial(trials_file, objective_func, space)


Using TensorFlow backend.


identity: model05
Attempt to resume a past training if it exists:
Found saved Trials! Loading...
Rerunning from 2 trials to add another one.
time: 2018-01-16 23:40:18.904199, identity: 8e361370-fad3-11e7-b733-525400f48026, params: {'split_dates': ('2016-01-01', '2017-01-01'), 'features': {'ma': {'ma_list': (1, 2, 3, 5, 8, 13, 21), 'window': (60,), 'price': 'close'}, 'label': {'class_list': (0.0, 1.0)}, 'kline2': {'window': 256}, 'label_by_multi_ma': {'window': (3, 5, 10)}}, 'lstm': {'activation_last': 'softmax', 'epochs': 1000, 'recurrent_dropout': 0.4, 'layer1': {'units': 128, 'is_BN': True, 'activation': 'tanh'}, 'bias_initializer': <keras.initializers.VarianceScaling object at 0x7fd9d1dce438>, 'kernel_initializer': <keras.initializers.VarianceScaling object at 0x7fd9d1dce240>, 'batch_size': 64, 'dropout': 0.4, 'shuffle': False, 'lr': 0.0001, 'loss_type': 'categorical_crossentropy', 'layer3': {'units': 128, 'is_BN': True, 'activation': 'tanh'}, 'layer2': {'units': 128, 'is_BN': True,

Epoch 50/1000
41/41 [==============================] - 0s 9ms/step - loss: 0.7464 - acc: 0.6532 - val_loss: 1.0507 - val_acc: 0.5104
Epoch 51/1000
41/41 [==============================] - 0s 8ms/step - loss: 0.8289 - acc: 0.6448 - val_loss: 1.0515 - val_acc: 0.5104
Epoch 52/1000
41/41 [==============================] - 0s 8ms/step - loss: 0.7891 - acc: 0.6273 - val_loss: 1.0530 - val_acc: 0.5104
Epoch 53/1000
41/41 [==============================] - 0s 9ms/step - loss: 0.7683 - acc: 0.6406 - val_loss: 1.0511 - val_acc: 0.5104
Epoch 54/1000
41/41 [==============================] - 0s 8ms/step - loss: 0.7882 - acc: 0.6280 - val_loss: 1.0490 - val_acc: 0.5104
Epoch 55/1000
41/41 [==============================] - 0s 8ms/step - loss: 0.7892 - acc: 0.6204 - val_loss: 1.0513 - val_acc: 0.5104
Epoch 56/1000
41/41 [==============================] - 0s 9ms/step - loss: 0.8325 - acc: 0.5983 - val_loss: 1.0500 - val_acc: 0.5104
Epoch 57/1000
41/41 [==============================] - 0s 8ms/step - 

Epoch 112/1000
41/41 [==============================] - 0s 8ms/step - loss: 0.6479 - acc: 0.6856 - val_loss: 1.0406 - val_acc: 0.5104
Epoch 113/1000
41/41 [==============================] - 0s 8ms/step - loss: 0.7237 - acc: 0.6479 - val_loss: 1.0404 - val_acc: 0.5104
Epoch 114/1000
41/41 [==============================] - 0s 10ms/step - loss: 0.7334 - acc: 0.6269 - val_loss: 1.0382 - val_acc: 0.5104
Epoch 115/1000
41/41 [==============================] - 0s 9ms/step - loss: 0.6773 - acc: 0.6822 - val_loss: 1.0419 - val_acc: 0.5104
Epoch 116/1000
41/41 [==============================] - 0s 8ms/step - loss: 0.6990 - acc: 0.6574 - val_loss: 1.0396 - val_acc: 0.5104
Epoch 117/1000
41/41 [==============================] - 0s 8ms/step - loss: 0.6900 - acc: 0.6665 - val_loss: 1.0403 - val_acc: 0.5104
Epoch 118/1000
41/41 [==============================] - 0s 8ms/step - loss: 0.7092 - acc: 0.6482 - val_loss: 1.0382 - val_acc: 0.5104
Epoch 119/1000
41/41 [==============================] - 0s 8m

41/41 [==============================] - 0s 8ms/step - loss: 0.7182 - acc: 0.6608 - val_loss: 1.0268 - val_acc: 0.5000
Epoch 174/1000
41/41 [==============================] - 0s 8ms/step - loss: 0.6637 - acc: 0.6528 - val_loss: 1.0245 - val_acc: 0.5000
Epoch 175/1000
41/41 [==============================] - 0s 9ms/step - loss: 0.6862 - acc: 0.6574 - val_loss: 1.0282 - val_acc: 0.5000
Epoch 176/1000
41/41 [==============================] - 0s 8ms/step - loss: 0.6786 - acc: 0.6456 - val_loss: 1.0296 - val_acc: 0.5000
Epoch 177/1000
41/41 [==============================] - 0s 8ms/step - loss: 0.6898 - acc: 0.6582 - val_loss: 1.0258 - val_acc: 0.4948
Epoch 178/1000

 log model into file ./params_select_model05/8e361370-fad3-11e7-b733-525400f48026/acc_0.7.h5....
41/41 [==============================] - 0s 9ms/step - loss: 0.6354 - acc: 0.7005 - val_loss: 1.0228 - val_acc: 0.4948
Epoch 179/1000
41/41 [==============================] - 0s 8ms/step - loss: 0.6780 - acc: 0.6860 - val_loss: 1.01

...

Epoch 729/1000
41/41 [==============================] - 0s 8ms/step - loss: 0.4964 - acc: 0.7553 - val_loss: 0.7132 - val_acc: 0.6927
Epoch 730/1000
41/41 [==============================] - 0s 8ms/step - loss: 0.5024 - acc: 0.7542 - val_loss: 0.7105 - val_acc: 0.6979
Epoch 731/1000
41/41 [==============================] - 0s 8ms/step - loss: 0.4928 - acc: 0.7805 - val_loss: 0.7093 - val_acc: 0.6979
Epoch 732/1000
41/41 [==============================] - 0s 7ms/step - loss: 0.4879 - acc: 0.7736 - val_loss: 0.7103 - val_acc: 0.6979
Epoch 733/1000
41/41 [==============================] - 0s 8ms/step - loss: 0.5053 - acc: 0.7641 - val_loss: 0.7087 - val_acc: 0.6979
Epoch 734/1000
41/41 [==============================] - 0s 8ms/step - loss: 0.4973 - acc: 0.7698 - val_loss: 0.7092 - val_acc: 0.6979
Epoch 735/1000
41/41 [==============================] - 0s 8ms/step - loss: 0.4972 - acc: 0.7576 - val_loss: 0.7102 - val_acc: 0.6979
Epoch 736/1000
41/41 [==============================] - 0s 8ms

Epoch 791/1000
41/41 [==============================] - 0s 8ms/step - loss: 0.5072 - acc: 0.7530 - val_loss: 0.7010 - val_acc: 0.6823
Epoch 792/1000
41/41 [==============================] - 0s 8ms/step - loss: 0.4879 - acc: 0.7713 - val_loss: 0.6975 - val_acc: 0.6875
Epoch 793/1000
41/41 [==============================] - 0s 8ms/step - loss: 0.4781 - acc: 0.7702 - val_loss: 0.6977 - val_acc: 0.6875
Epoch 794/1000
41/41 [==============================] - 0s 8ms/step - loss: 0.4991 - acc: 0.7595 - val_loss: 0.6984 - val_acc: 0.6875
Epoch 795/1000
41/41 [==============================] - 0s 8ms/step - loss: 0.5025 - acc: 0.7462 - val_loss: 0.6955 - val_acc: 0.6875
Epoch 796/1000
41/41 [==============================] - 0s 8ms/step - loss: 0.4831 - acc: 0.7668 - val_loss: 0.6977 - val_acc: 0.6927
Epoch 797/1000
41/41 [==============================] - 0s 8ms/step - loss: 0.5187 - acc: 0.7595 - val_loss: 0.7012 - val_acc: 0.6823
Epoch 798/1000
41/41 [==============================] - 0s 9ms

Epoch 853/1000
41/41 [==============================] - 0s 9ms/step - loss: 0.4929 - acc: 0.7645 - val_loss: 0.7101 - val_acc: 0.6771
Epoch 854/1000
41/41 [==============================] - 0s 8ms/step - loss: 0.5048 - acc: 0.7652 - val_loss: 0.7078 - val_acc: 0.6771
Epoch 855/1000
41/41 [==============================] - 0s 8ms/step - loss: 0.4851 - acc: 0.7752 - val_loss: 0.7071 - val_acc: 0.6771
Epoch 856/1000

 log model into file ./params_select_model05/8e361370-fad3-11e7-b733-525400f48026/acc_0.8.h5....
41/41 [==============================] - 0s 8ms/step - loss: 0.4471 - acc: 0.8007 - val_loss: 0.7051 - val_acc: 0.6771
Epoch 857/1000
41/41 [==============================] - 0s 7ms/step - loss: 0.4758 - acc: 0.7862 - val_loss: 0.7027 - val_acc: 0.6823
Epoch 858/1000
41/41 [==============================] - 0s 8ms/step - loss: 0.5048 - acc: 0.7683 - val_loss: 0.7017 - val_acc: 0.6823
Epoch 859/1000
41/41 [==============================] - 0s 8ms/step - loss: 0.4807 - acc: 0.7725 -

Epoch 914/1000
41/41 [==============================] - 0s 8ms/step - loss: 0.4709 - acc: 0.7862 - val_loss: 0.7132 - val_acc: 0.6562
Epoch 915/1000
41/41 [==============================] - 0s 8ms/step - loss: 0.4713 - acc: 0.7839 - val_loss: 0.7152 - val_acc: 0.6562
Epoch 916/1000
41/41 [==============================] - 0s 8ms/step - loss: 0.4723 - acc: 0.7843 - val_loss: 0.7126 - val_acc: 0.6615
Epoch 917/1000
41/41 [==============================] - 0s 7ms/step - loss: 0.4798 - acc: 0.7706 - val_loss: 0.7137 - val_acc: 0.6615
Epoch 918/1000
41/41 [==============================] - 0s 7ms/step - loss: 0.4896 - acc: 0.7748 - val_loss: 0.7117 - val_acc: 0.6615
Epoch 919/1000
41/41 [==============================] - 0s 8ms/step - loss: 0.4915 - acc: 0.7641 - val_loss: 0.7100 - val_acc: 0.6615
Epoch 920/1000
41/41 [==============================] - 0s 8ms/step - loss: 0.4613 - acc: 0.7835 - val_loss: 0.7067 - val_acc: 0.6615
Epoch 921/1000
41/41 [==============================] - 0s 7ms

Epoch 976/1000
41/41 [==============================] - 0s 9ms/step - loss: 0.4492 - acc: 0.7988 - val_loss: 0.7013 - val_acc: 0.6667
Epoch 977/1000
41/41 [==============================] - 0s 9ms/step - loss: 0.4793 - acc: 0.7771 - val_loss: 0.7004 - val_acc: 0.6667
Epoch 978/1000
41/41 [==============================] - 0s 8ms/step - loss: 0.4575 - acc: 0.7896 - val_loss: 0.7026 - val_acc: 0.6667
Epoch 979/1000
41/41 [==============================] - 0s 8ms/step - loss: 0.4608 - acc: 0.8011 - val_loss: 0.7015 - val_acc: 0.6667
Epoch 980/1000
41/41 [==============================] - 0s 8ms/step - loss: 0.4548 - acc: 0.7828 - val_loss: 0.7022 - val_acc: 0.6667
Epoch 981/1000
41/41 [==============================] - 0s 7ms/step - loss: 0.4353 - acc: 0.8117 - val_loss: 0.7032 - val_acc: 0.6719
Epoch 982/1000
41/41 [==============================] - 0s 7ms/step - loss: 0.4634 - acc: 0.7843 - val_loss: 0.7045 - val_acc: 0.6719
Epoch 983/1000
41/41 [==============================] - 0s 7ms